In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from gensim.models import Word2Vec
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
from sklearn.metrics import make_scorer, f1_score
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

In [2]:
# Assuming your data is in a CSV file named 'movie_data.csv'
data = pd.read_csv('movies_initial.csv')

data = data[['genre','fullplot']]

data = data.dropna(subset=['genre', 'fullplot'])
print(data)

                          genre  \
0            Documentary, Short   
1                         Short   
2      Animation, Comedy, Short   
3            Documentary, Short   
4            Documentary, Short   
...                         ...   
46004                     Drama   
46006               Documentary   
46007                    Horror   
46008  Comedy, Fantasy, Romance   
46010                    Sci-Fi   

                                                fullplot  
0      Performing on what looks like a small wooden s...  
1      A stationary camera looks at a large anvil wit...  
2      One night, Arlequin come to see his lover Colo...  
3      A man (Edison's assistant) takes a pinch of sn...  
4      A man opens the big gates to the Lumi�re facto...  
...                                                  ...  
46004  A post modern theater adaptation of a classic ...  
46006  Musician Jonny Greenwood travels to Rajasthan,...  
46007  A cash strapped student who starts workin

In [3]:

print("Step 1: Data Preprocessing")
# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
print("Training Data Shape:", train_data.shape)
print("Testing Data Shape:", test_data.shape)

print("\nWord2Vec model training on the movie plot text")
# Word2Vec model training on the movie plot text
word2vec_model = Word2Vec(sentences=train_data['fullplot'].apply(lambda x: x.split()), vector_size=100, window=5, min_count=1, workers=4)


Step 1: Data Preprocessing
Training Data Shape: (32995, 2)
Testing Data Shape: (8249, 2)

Word2Vec model training on the movie plot text


In [4]:
# Function to calculate the average word vector for a text
def average_word_vector(words, model, num_features):
    feature_vector = np.zeros((num_features,), dtype="float32")
    n_words = 0
    for word in words:
        if word in model.wv:
            n_words += 1
            feature_vector = np.add(feature_vector, model.wv[word])
    if n_words > 0:
        feature_vector = np.divide(feature_vector, n_words)
    return feature_vector

In [ ]:

print("\nCreate word vectors for training and testing sets")
# Create word vectors for training and testing sets
X_train_word2vec = np.array([average_word_vector(plot.split(), word2vec_model, 100) for plot in tqdm(train_data['fullplot'])])
X_test_word2vec = np.array([average_word_vector(plot.split(), word2vec_model, 100) for plot in tqdm(test_data['fullplot'])])

print("Word Vector Shape - Training:", X_train_word2vec.shape)
print("Word Vector Shape - Testing:", X_test_word2vec.shape)


print("\nTransform genres into binary matrix for multi-label classification")
# Transform genres into binary matrix for multi-label classification
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(train_data['genre'].apply(lambda x: set(x.split(', '))))
y_test = mlb.transform(test_data['genre'].apply(lambda x: set(x.split(', '))))


print(y_train)


Create word vectors for training and testing sets


 61%|████████████████████████████████████████████▎                            | 20011/32995 [00:01<00:01, 12261.14it/s]

In [ ]:
# Set the max_depth parameter
max_depth_value = 20  # Adjust this value based on your experimentation

print("\nStep 2: Random Forest Model Training with Max Depth")
# Random Forest model training with tqdm and max_depth
rf_model_max_depth = RandomForestClassifier(n_estimators=150, max_depth=max_depth_value, random_state=42)

# Fit the model
rf_model_max_depth.fit(X_train_word2vec, y_train)

print("\nStep 3: Evaluation with Max Depth")
# Predict the labels for the test set
y_pred_rf_max_depth = rf_model_max_depth.predict(X_test_word2vec)

# Evaluate the performance
accuracy_rf = accuracy_score(y_test, y_pred_rf_max_depth)
print(f"Random Forest Accuracy: {accuracy_rf:.2f}")

# Display detailed classification report for Random Forest with Max Depth
print("\nRandom Forest Classification Report with Max Depth:")
print(classification_report(y_test, y_pred_rf_max_depth,target_names=mlb.classes_))

In [ ]:
print(y_pred_rf_max_depth[:20],y_test[:20])
array1 = np.array(y_pred_rf_max_depth)
array2 = np.array(y_test)
matching_ones = np.sum((array1 == 1) & (array2 == 1))
not_matching_ones = np.sum((array1 == 1) & (array2 == 0) | (array1 == 0) & (array2 == 1))

# Print the results
print("Number of matching 1s:", matching_ones)
print("Number of not matching 1s:", not_matching_ones)
print('')